In [2]:
from redpitaya.overlay.mercury import mercury as overlay
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path

#self.FPGA = overlay() #hier einfach aus der Klasse gezogen, vielleicht reicht es

#otherwise 
#if
#'overlay' in dir ()

In [ ]:
class SchwarzerStrahler:
    
    def __init__(self, messanzahl=1000):
        print("Hello there")
        #self.FPGA = overlay()
        #Make an instance of the oscilloscope, the first argument is the channel index 
        #in range [0,1]. The second argument is the voltage range in volts, 
        #should be either 1.0 (default if unspecified) or 20.0.
        self.osci = self.FPGA.osc(0, 20.0) # 20 oder 1 V
        
        # data rate decimation 
        # is an integer in range [1, 2**17] = [1, 131072].
        self.osci.decimation = 1

        # trigger timing [sample periods]
        # Number of samples stored before and after trigger, 
        #for a forced trigger only post trigger samples are enough.
        self.trig = messanzahl
        self.osci.trigger_pre  = 0
        self.osci.trigger_post = self.trig
        
        # disable hardware trigger sources
        self.osci.trig_src = 0
        
        print("Fertig Initialisiert")

     #### Funzionen

    def measure(self):
        self.osci.reset()
        self.osci.start()
        self.osci.trigger()
        while (self.osci.status_run()):
            pass
        return self.osci.data(self.trig)

In [ ]:
Experiment = SchwarzerStrahler()

Im Abstand von 0.5 Sekunden wird jeweils eine Messung ausgelöst. Und an eine Liste gehängt.
'Elapsed Time' gibt die Anzahl an vergangege Sekunden an.

Eine Vollständige Bewegung des Schrittmotors von Anfang bis Ende dauert ca. 4.3 Minuten = 258 Sekunden.

In [9]:
stepliste = []
messwerte = []
timelist = []
startTime = time.time()

with open("OsziMessTest11.csv", "w", encoding="utf-8", buffering=1) as fs:      
# Datei: erste spalte sind schritte, zweite sind die Messewrte dann Zeilenumbruch.
        fs.write("Schritt,Messwert,{}\n".format(np.mean(Experiment.measure())))    
            # Hier messen und jedes mal den Mittelwert ausgeben
        for i in range(500): #500 ca. Schritte sind für eine Messung gut. 
            time.sleep(0.5)
            timelist.append(time.time())
            stepliste.append(i)
            fs.write("{},{}\n".format(i, np.mean(Experiment.measure())))
            print(str(100 * i / 500 ) + "%", end="\r")
            messwerte.append(np.mean(Experiment.measure()))
            endTime = time.time()
            elapsedTime = endTime - startTime
            print("Elapsed Time = %i sec." % elapsedTime,end = "\r")


In [ ]:
plt.plot(stepliste,messwerte)
plt.show()